In [ ]:
# https://www.perplexity.ai/search/give-me-some-example-queries-f-4r_tr4.qTCuChf7GWRiUvQ

In [16]:
# Here we define our query as a multi-line string
query = '''
query ($id: Int) { # Define which variables will be used in the query (id)
  Media (id: $id, type: ANIME) { # Insert our variables into the query arguments (id) (type: ANIME is hard-coded in the query)
    id
    title {
      romaji
      english
      native
    }
  }
}
'''

# Define our query variables and values that will be used in the query request
variables = {
    'id': 1512
}

In [24]:
query = '''
query ($search: String) {
  Page(page: 1, perPage: 10) {
    media(search: $search, type: ANIME) {
      id
      title {
        romaji
        english
      }
      genres
    }
  }
}

'''

variables = {
  "search": "Attack on Titan" # seems to work with non exact matches
}


In [20]:
query = '''
query ($page: Int, $perPage: Int) {
  Page(page: $page, perPage: $perPage) {
    media(type: ANIME, sort: POPULARITY_DESC) {
      id
      title {
        romaji
        english
      }
      popularity
    }
  }
}
'''

variables = {
  "page": 1,
  "perPage": 5
}


In [ ]:
import requests


url = 'https://graphql.anilist.co'

# Make the HTTP Api request
response = requests.post(url, json={'query': query, 'variables': variables})

print(response.json())

In [ ]:
from crewai.tools import BaseTool
from pydantic import BaseModel, Field

In [15]:
import requests
import os
from crewai import Agent, Task, Crew, Process

from langchain_openai import ChatOpenAI

# Set up the LLM (OpenAI GPT-3.5 Turbo)
api_key = os.getenv("OPENAI_API_KEY")
if not api_key:
    raise ValueError("Please set the OPENAI_API_KEY environment variable.")
llm = ChatOpenAI(model="gpt-4o-mini", api_key=api_key)

# Custom tool to query the AniList API
class AniListAPITool(BaseTool):
    name: str = "AniListAPITool"
    description: str = "Queries the AniList GraphQL API to fetch anime recommendations based on a search term."
    
    def _run(self, search_term: str) -> list:
        # GraphQL query to search for anime
        query = """
        query ($search: String, $type: MediaType, $sort: [MediaSort], $perPage: Int) {
          Page(page: 1, perPage: $perPage) {
            media(search: $search, type: $type, sort: $sort) {
              id
              title {
                romaji
                english
              }
            }
          }
        }
        """
        variables = {
            "search": search_term,
            "type": "ANIME",
            "sort": ["SEARCH_MATCH"],
            "perPage": 5
        }
        url = "https://graphql.anilist.co"
        response = requests.post(url, json={"query": query, "variables": variables})

        # Parse the response
        if response.status_code == 200:
            data = response.json()
            media_list = data.get("data", {}).get("Page", {}).get("media", [])
            # Use English title if available, otherwise Romaji
            titles = [media["title"]["english"] or media["title"]["romaji"] for media in media_list]
            return titles
        return []  # Return empty list if the API call fails

In [40]:
import requests
import os
from crewai import Agent, Task, Crew, Process
from crewai.tools import BaseTool
from langchain_openai import ChatOpenAI

# Set up the LLM (OpenAI GPT-3.5 Turbo)
api_key = os.getenv("OPENAI_API_KEY")
if not api_key:
    raise ValueError("Please set the OPENAI_API_KEY environment variable.")
llm = ChatOpenAI(model="gpt-3.5-turbo", api_key=api_key)

# Custom tool to query the AniList API
class AniListAPITool(BaseTool):
    name: str = "AniListAPITool"
    description: str = "Queries the AniList GraphQL API to fetch anime recommendations based on a search term."

    def _run(self, search_term: str) -> list:
        query = """
        query ($search: String, $type: MediaType, $sort: [MediaSort], $perPage: Int) {
          Page(page: 1, perPage: $perPage) {
            media(search: $search, type: $type, sort: $sort) {
              id
              title {
                romaji
                english
              }
            }
          }
        }
        """
        variables = {
            "search": search_term,
            "type": "ANIME",
            "sort": ["SEARCH_MATCH"],
            "perPage": 5
        }
        url = "https://graphql.anilist.co"
        response = requests.post(url, json={"query": query, "variables": variables})
        if response.status_code == 200:
            data = response.json()
            media_list = data.get("data", {}).get("Page", {}).get("media", [])
            titles = [media["title"]["english"] or media["title"]["romaji"] for media in media_list]
            return titles
        return []

# Agent 1: Preference Extractor
preference_extractor = Agent(
    role="Preference Extractor",
    goal="Extract key search terms from the user's anime recommendation request.",
    backstory="You're a pro at understanding what people want when they ask for anime recommendations. You pick out genres, themes, or specific titles.",
    llm=llm,
    verbose=True
)

# Agent 2: AniList Searcher
anilist_searcher = Agent(
    role="AniList Searcher",
    goal="Query the AniList API with the search term and provide anime recommendations.",
    backstory="You're an expert at digging through the AniList API to find the best anime matches for any search term.",
    tools=[AniListAPITool()],
    llm=llm,
    verbose=True
)

# Task 2: Search AniList with the extracted term
search_anilist = Task(
    description='Use the search term from the previous task to query the AniList API with AniListAPITool. '
                'Fetch 5 anime titles and format them into a message like "Here are some recommendations based on \'[search_term]\':\\n1. Title1\\n2. Title2\\n...".',
    agent=anilist_searcher,
    context=[extract_search_criteria],
    expected_output="A formatted string with anime recommendations."
)

# Task 1: Extract the search term with the user's actual request
extract_search_criteria = Task(
    description=f'Extract the key search term from this user request: "{user_request}". '
                'The key term is either a genre, a theme, or a specific anime title. '
                'Focus on identifying the genre or title that defines the users preference. '
                'For example:\n'
                '- If the user says "recommend a horror anime", extract "horror".\n'
                '- If the user says "find anime similar to \'Naruto\'", extract "Naruto".\n'
                '- If the user says "suggest some action-packed anime", extract "action-packed".\n'
                '- If the user says "I want to watch a romance anime", extract "romance".\n'
                '- If the user says "recommend a funny slice-of-life anime", extract "funny slice-of-life".\n'
                'Return the extracted term as a string.',
    agent=preference_extractor,
    expected_output="A string with the search term."
)

In [44]:
crew = Crew(
        agents=[preference_extractor, anilist_searcher],
        tasks=[extract_search_criteria, search_anilist],
        process=Process.sequential,
        verbose=False
    )

user_request = "I want to watch a ecchi anime"
result = crew.kickoff(inputs={"user_request": user_request})
print(result)

# Agent: Preference Extractor
## Task: Extract the key search term from this user request: "I want to watch a ecchi anime". The key term is either a genre, a theme, or a specific anime title. Focus on identifying the genre or title that defines the users preference. For example:
- If the user says "recommend a horror anime", extract "horror".
- If the user says "find anime similar to 'Naruto'", extract "Naruto".
- If the user says "suggest some action-packed anime", extract "action-packed".
- If the user says "I want to watch a romance anime", extract "romance".
- If the user says "recommend a funny slice-of-life anime", extract "funny slice-of-life".
Return the extracted term as a string.


# Agent: Preference Extractor
## Final Answer: 
ecchi


# Agent: AniList Searcher
## Task: Use the search term from the previous task to query the AniList API with AniListAPITool. Fetch 5 anime titles and format them into a message like "Here are some recommendations based on '[search_term]':\n1. T